## 2표본 문제에 관한 가설검정

두 모집단에 대한 검정이다. 

* 독립표본의 평균 비교(Two-sample t-test)
    * 서로 다른(서로 독립적인) 두 집단의 평균의 차이를 분석하는 검정이다.
* 대응표본의 평균 비교(Paired t-test)
    * 한 집단 표본을 사전과 사후로 나눠서 2회 측정한 후 두 측정치 간 차이 여부를 분석하는 검정이다.

In [4]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

%precision 3
np.random.seed(1111)

### 1. 독립비교 t검정

독립된 두 집단의 평균 차이가 있는지를 검사하는 방법이다.

* H0 : $\mu_1 = \mu_2$
* H1 : $\mu_1 \ne \mu_2$

다음의 예제는 A반과 B반 학생의 집중력 테스트와 관련된 자료이다. 이 자료를 통해서 A반과 B반의 집중력에 유의한 차이가 있는지 검정해 보자.

In [2]:
training_ind = pd.read_csv('./data/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


#### stats.ttest_ind 함수

이 경우 다음과 같은 가설검정을 수행한다. 

* H0 : $\mu_1 - \mu_2 = 0$
* H1 : $\mu_1 - \mu_2 \ne 0$

검정통계량은 다음과 같다.
$$ T = \dfrac{(\bar{X} - \bar{Y}) - (\mu_1 - \mu_2)}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}$$

stats.ttest_ind 함수를 사용하면 간단하게 Two sample t-test를 시행할 수 있다. 이때 두 모집단의 분산은 같다는 가정하에 수행한다. 그러나 만약 두 모집단의 분산이 같다는 가정이 만족하지 않는다면 `stats.ttest_ind`함수에서 `equal_var=False`를 설정하여 수행한다.(기본은 `equal_var=True`)   

이 함수는  t검정 통계량과 p-value를 리턴한다.

In [3]:
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'],
                       equal_var=True)
p

0.08631600572195147

p-value가 유의수준 0.05보다 크기 때문에 귀무가설을 기각할 수 없다. 즉, A반과 B반의 집중력 평균 점수의 유의한 차이가 있다고 할 수 없다.

### 2. 대응비교 t검정 paird t-test

대응표본은 두 데이터에서 서로 대응하는 동일한 개체에 대해 각 다른 조건으로 측정한 것을 말한다. 예를 들면 약을 투약하기 전/후에 측정한 혈압은 동일 피검자에 대해 투약 전/후라는 조건으로 측정한 것으로 대응표본이다. 따라서 이때 해당 약물이 혈압을 상승시키는 효과가 있는지 확인하려면 대응표본의 대표값 차이에 대한 검정을 수행해야 한다. 

다음과 같이 특정 근력운동이 집중력 향상에 도움이 되는지 검정하고자 한다. 근력 운동 전/후에 집중력 테스트를 시행하였다. 근력운동이 집중력 향상에 유의한 차이를 가져오는지 검정해 본다. 

In [4]:
training_rel = pd.read_csv('./data/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


* 귀무가설 H0 : $\mu_{after} - \mu_{before} = 0$ 
* 대립가설 H1 : $\mu_{after} - \mu_{before} \ne 0$

각 데이터의 차이를 구해보면 다음과 같다.

In [5]:
training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


이 가설검정에서는 차이의 평균을 $\mu_{diff}$로 하여 
* H0 : $\mu_{diff} = 0 $
* H1 : $\mu_{diff} \ne 0 $
로 바꿔 말할 수 있다. 따라서 앞서 수행한 독립변수에 대한 일표본 t-검정을 수행할 수 있다.

In [6]:
t, p = stats.ttest_1samp(training_rel['차'], 0)
p

0.04004419061842953

p-value가 유의수준 0.05보다 작으므로 귀무가설을 기각한다. 즉 근력운동 전/후로 집중력의 유의한 차이가 있다. 

#### scipy.ttest_rel 함수

scipy.ttest_rel 함수를 사용하면 대응표본에 대한 t검정을 손쉽게 수행할 수 있다.   
이 함수를 사용하면 일일히 before/after의 차이를 구하지 않아도 되므로 실제 데이터에 대한 검정에서는 이 함수를 더 많이 사용한다.

In [7]:
t, p = stats.ttest_rel(training_rel['후'], training_rel['전'])
p

0.04004419061842953

# 실습

1. 다음과 같은 데이터가 있다. 특정약을 복용하기 전/후의 체온이 차이가 있는지 검정해 보자.

```
data = pd.DataFrame({'medicine':['before', 'before', 'before', 'before', 'before', 'after', 'after','after', 'after', 'after'],
                     'body_temperature':[36.2, 36.2, 35.3, 36.1, 36.1, 36.8, 36.1, 36.8, 37.1, 36.9]})
                 
data                 
```
